In [1]:
import numpy as np
import scipy.io as sio
from scipy.linalg import toeplitz
from scipy.stats import multivariate_normal
import time
import random
from scipy.linalg import block_diag
from sklearn.metrics import precision_recall_curve, auc
import scipy.stats
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import random
import scipy.sparse as sp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pickle
from collections import defaultdict

import sys
# sys.path.append('/Users/yijwang-admin/Documents/Research/GFL/Code/PQN_Python_main')
from minConf.minConf_PQN import minConF_PQN
import random
import itertools
import os
import matlab.engine

# Section 1: Data Generation

Here we generate data for random ensemble I.

To generate the synthetic data, we need to generate the following: the design matrix $X$, the graph (there are several equivalent way to record the information, adjacent matrix $A$, Laplacian $L$, etc.), the weight $w$, and the response vector $y$.

Given the sample size $n$, the number of features $d$, the random design matrix $X \in \mathbb{R}^{n \times d} $ is generated with i.i.d. $ \mathcal{N}(0, 1) $ entries. 

Given the number of selected features $k$, without loss of generality (by symmetry and permutation), we choose the first $k$ as the selected features. i.e. non-zero features contributing to the target. We randomly partition this $k$ features into $h'$ clusters with size $c_1, c_2, \dots, c_{h'}$. We also randomly partition the rest features into $h''$ clusters with size $c_{h'+1}, c_{h'+2}, \dots, c_{h'+h''}$. Obviously, we have $c_1 + \dots + c_{h'} = k$  and $c_{h'+1}+ \dots c_{h'+h''}= d-k$. We denote the total number of the clusters by $h  = h' + h''$.

We construct the regression weight vector \( w \in \mathbb{R}^d \) such that non-zero coefficients only appears for the selected feature. For features in the same cluster, the corresponding coefficients in $w$ are set to be the same. We randomly assign them from one of the value $\pm \frac{1}{\sqrt{k}}$. 

The next step is to generate the graph which is compatible with the cluster structure with some noise. Fix two probability parameters $p > \frac{1}{2}$ and $q < \frac{1}{2}$, which indicates that probability of edges connecting two feature nodes in a same cluster and probability of edges connecting two feature nodes in different cluster, respectively. It reflect that there are some edge missing and some noisy edges in the real world prior graphical knowledge. In other words, we generate the graph by connecting two nodes in a same cluster with probability $p$ and connecting two nodes in different cluster with probability $q$.

Finally, we define the response vector \( y = X w + \epsilon \), where the noise vector \( \epsilon \in \mathbb{R}^n \) has i.i.d. \( \mathcal{N}(0, \gamma^2) \) entries. The goal is to identify the support of \( w \) and the corresponding coefficients. This simulation design allows us to examine whether the method can recover (the cluster structure) and individual feature sparsity under this random ensemble.

We need following parameters to determine a random ensemble:
- `n`: the number of samples
- `d`: the number of features
- `k`: the number of selected features
- `h_total`: the number of clusters
- `h_selected`: the number of clusters for the selected features
- `h_rest`: the number of clusters for the rest features
- `gamma`: the noise level
- `p`: the probability of edges connecting two feature nodes in a same cluster
- `q`: the probability of edges connecting two feature nodes in different cluster

## Section 1.1 Graph Generation

In [2]:
import numpy as np
from scipy.sparse import spdiags
from scipy.linalg import inv
from gurobipy import Model, GRB, QuadExpr

def Lasso(u, X, y, rho):
    n, d = X.shape
    D_u = spdiags(u.flatten(), 0, d, d)
    M = inv((1/rho) * X @ D_u @ X.T + np.eye(n))
    f = y.T @ M @ y
    g = -(1/rho) * ((X.T @ M @ y)**2)

    return f, g

def GroupLasso(u, X, y, rho):
    n, d = X.shape
    D_u = spdiags(u.flatten(), 0, d, d)
    M = inv((1/rho) * X @ D_u @ X.T + np.eye(n))
    f = y.T @ M @ y
    g = -(1/(2*rho)) * ((X.T @ M @ y)**2)

    return f, g

def GeneralizedFusedLasso(u, X, y, rho, L, mu):
    n, d = X.shape
    D_u = spdiags(u.flatten(), 0, d, d)
    M = inv((1/rho) * X @ D_u @ X.T + np.eye(n))
    f = y.T @ M @ y + mu * u.T @ L @ u
    g = -(1/(2* rho)) * ((X.T @ M @ y)**2) + 2 * mu * L @ u

    return f.flatten(), g.flatten()


def ProjLassoGurobi(u, k, d):
    A = np.ones((1, d))
    b = np.array([k])
    Q = np.eye(d)
    f = -2 * u.flatten()

    model = Model("GeneralizedFusedLasso")
    x = model.addMVar(d, lb=0.0, ub=1.0)

    model.setObjective(x@Q@x + x@f)
    model.addConstr(A @ x == b)

    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    model.optimize()
    # print("Lasso x.x: ", x.x)

    return x.x

def ProjGroupLassoGurobi(u, k, groups, h, d):
    """
    Solves the Group Lasso projection problem using Gurobi.

    Parameters:
    - u: Input vector of size (d, 1)
    - k: Sparsity level (L1 constraint)
    - groups: List of groups, where each group is a list of indices
    - h: Sum constraint for group norms

    Returns:
    - up: Projected vector (u')
    - zp: Auxiliary variables for group norms
    """
    # d = u.shape  # Number of variables
    g = len(groups)  # Number of groups

    # Quadratic objective matrices and linear term
    H1 = np.eye(d)  # Identity for quadratic term on `u`
    f1 = -2 * u.flatten()  # Linear term for `u`
    H2 = np.zeros((g, g))  # No quadratic term for auxiliary variables
    f2 = np.zeros(g)  # No linear term for auxiliary variables

    # Combine H1, H2 and f1, f2 into block matrices
    H = np.block([[H1, np.zeros((d, g))], [np.zeros((g, d)), H2]])
    f = np.concatenate([f1, f2])

    # Constraint matrices for `u` (A1) and `z` (A2)
    A1 = np.vstack([np.ones((1, d)), -np.ones((1, d))])  # L1 constraint on `u`
    A2 = np.zeros((2, g))  # Initialize A2
    group_constraints_count = 0

    # Build group constraints
    for i, group in enumerate(groups):
        for idx in group:
            A1_row = np.zeros((1, d))
            A1_row[0, idx] = 1
            A1 = np.vstack([A1, A1_row])  # Adding constraints for group indices

            A2_row = np.zeros((1, g))
            A2_row[0, i] = -1
            A2 = np.vstack([A2, A2_row])  # Adding constraints for group norms
            group_constraints_count += 1

    # Add overall group norm constraint
    A1 = np.vstack([A1, np.zeros((1, d))])
    A2 = np.vstack([A2, np.ones((1, g))])
    A = np.hstack([A1, A2])  # Combine A1 and A2

    # RHS vector for constraints
    b = np.zeros(group_constraints_count + 3)
    b[0] = k  # Sparsity constraint (sum of u <= k)
    b[1] = -k + 1  # Lower bound for sparsity
    b[-1] = h  # Sum constraint for group norms

    # Lower and upper bounds for variables
    lb = np.zeros(d + g)
    ub = np.ones(d + g)

    # Setup Gurobi model
    model = Model("ProjGroupLasso")
    x = model.addMVar(d + g, lb=lb, ub=ub)  # Variables for `u` and `z`

    # Set quadratic objective
    model.setObjective(x @ H @ x + x @ f)

    # Add constraints
    model.addConstr(A @ x <= b)

    # Configure Gurobi parameters
    model.setParam('OutputFlag', 0)  # Suppress output
    model.setParam('IterationLimit', 500)
    # Solve the model
    model.optimize()

    # Extract the results
    up = x.x[:d]  # Projected vector `u'`
    zp = x.x[d:]  # Auxiliary variables `z'`

    # print("GroupLasso x.x: ", up[:, np.newaxis])

    return up


def ProjGeneralizedFusedLassoGurobi(u, k, d):
    A = np.ones((1, d))
    b = np.array([k])
    Q = np.eye(d)
    f = -2 * u.flatten()

    model = Model("GeneralizedFusedLasso")
    x = model.addMVar(d, lb=0.0, ub=1.0)

    model.setObjective(x@Q@x + x@f)
    model.addConstr(A @ x == b)

    model.setParam('OutputFlag', 0)
    model.setParam('IterationLimit', 500)

    model.optimize()
    # print("GeneralizedFusedLasso x.x: ", x.x)

    return x.x

In [4]:
class RandomEnsemble:
    def __init__(self, n, d, k, h_total, h_selected, h_rest, gamma, 
                 p=0.95, q=0.01, 
                 options=None, num_replications=20,
                 datafile=f'./code_fgfl_aaai14/data_gfl/',
                 resultfile='./code_fgfl_aaai14/result_gfl/'):
        assert h_total == h_selected + h_rest, "h_total should be equal to h_selected + h_rest"
        self.n = n
        self.d = d
        self.k = k
        self.h_total = h_total
        self.h_selected = h_selected
        self.h_rest = h_rest
        self.gamma = gamma
        self.p = p
        self.q = q
        self.options = options if options else {'maxIter': 500, 'verbose': 0}
        self.num_replications = num_replications
        self.datafile = os.path.join(os.path.abspath(datafile), f'{n}_{d}_{k}_{h_total}_{h_selected}_{h_rest}_{gamma}_{p}_{q}')
        self.resultfile = os.path.join(os.path.abspath(resultfile), f'{n}_{d}_{k}_{h_total}_{h_selected}_{h_rest}_{gamma}_{p}_{q}') # matlab does not like relative path
        self.best_rho1 = 0.5
        self.best_rho2 = 0.5
        self._init(self.datafile, self.resultfile)
    
    def _init(self, datafile, resultfile):
        if not os.path.exists(datafile):
            os.makedirs(datafile)
        if not os.path.exists(resultfile):
            os.makedirs(resultfile)

    def _random_partition(self):
        # partition the first k nodes into h_selected groups and the rest into h_rest groups
        assert self.h_selected <= self.k, "h_selected should be less than k"
        assert self.h_rest <= self.d - self.k, "h_rest should be less d-k"
        break_points = np.sort(random.sample(range(1, self.k), self.h_selected-1))
        break_points_rest = np.sort(random.sample(range(self.k+1, self.d), self.h_rest-1))

        return break_points, break_points_rest
    
    def _generate_clusters(self):
        break_points, break_points_rest = self._random_partition()
        clusters = []
        clusters.append(np.arange(break_points[0])) # first selected cluster
        for i in range(1, self.h_selected-1):
            clusters.append(np.arange(break_points[i-1], break_points[i]))
        clusters.append(np.arange(break_points[-1], self.k)) # last selected cluster

        clusters.append(np.arange(self.k, break_points_rest[0])) # first rest cluster
        for i in range(1, self.h_rest-1):
            clusters.append(np.arange(break_points_rest[i-1], break_points_rest[i]))
        clusters.append(np.arange(break_points_rest[-1], self.d))

        return clusters

    def _generate_graph(self):
        # here we generate the adjacency matrix and laplacian of the graph
        clusters = self._generate_clusters()

        A = sp.lil_matrix((self.d, self.d))
        
        # generate thr inner cluster connections
        for cluster in clusters:
            cluster_size = len(cluster)
            block = (np.random.rand(cluster_size, cluster_size) < self.p).astype(int)
            np.fill_diagonal(block, 0) # no self-loop
            block = np.triu(block) + np.triu(block, 1).T # make it symmetric
            for i, node_i in enumerate(cluster):
                for j, node_j in enumerate(cluster):
                    A[node_i, node_j] = block[i, j]

        # generate the connections between clusters
        for i in range(self.h_total):
            for j in range(i+1, self.h_total):
                cluster_i = clusters[i]
                cluster_j = clusters[j]
                block = (np.random.rand(len(cluster_i), len(cluster_j)) < self.q).astype(int)
                for m, node_i in enumerate(cluster_i):
                    for n, node_j in enumerate(cluster_j):
                        A[node_i, node_j] = block[m, n]
                        A[node_j, node_i] = block[m, n]

        # TODO: check if the graph is connected and make it connected if not 
        # (optional, maybe not necessary)
        D = sp.diags(np.ravel(A.sum(axis=1)))
        L = D - A
        return L, clusters, A
    
    def _visualize_graph(self, A):
        # if we want to visualize the graph, we need to change it to array rather than sparse matrix
        A_arr = A.toarray()
        plt.figure(figsize=(8, 8))
        plt.title('Adjacency matrix')
        plt.spy(A_arr)
        plt.axis('off')
        plt.show()

    def _generate_w(self, clusters):
        w = np.zeros(self.d)
        for i in range(self.h_selected):
            cluster_sign = np.random.choice([-1, 1])
            cluster_weight = cluster_sign * (1 / np.sqrt(self.k)) # TODO: make more choice other than 1/sqrt(k)
            for node in clusters[i]:
                w[node] = cluster_weight
        # w = w[:, np.newaxis]
        return w

    def _generate_X(self):
        X = np.random.normal(0, 1, (self.n, self.d))
        return X
    
    def _generate_y(self, X, w):
        signal = X @ w
        noise = np.random.normal(0, self.gamma, signal.shape)
        y = signal + noise
        SNR = self._compute_snr(signal, noise)
        if 0: # TODO: add a debug/verbose flag
            print(f'SNR: {SNR}')

        return y

    def _generate_data(self):
        L, clusters, A = self._generate_graph()
        if 0: # TODO: add a debug/verbose flag 
            self._visualize_graph(A)
        w = self._generate_w(clusters)
        X = self._generate_X()
        y = self._generate_y(X, w)
        return L, w, X, y, clusters, A


    def _compute_snr(self, signal, noise):  
        # in our case, the snr is 10*log10(1/gamma^2)
        signal = np.asarray(signal)
        noise = np.asarray(noise)

        signal_power = np.mean(signal ** 2)
        noise_power = np.mean(noise ** 2)
        
        snr = signal_power / noise_power
        snr_db = 10 * np.log10(snr)
        
        return snr_db
    
    def _define_obj(self, model, X, y, rho=1, L=None, mu=None):
        """
        Support models: 'Lasso', GroupLasso', 'GeneralizedFusedLasso'
        I plan to support Proximal method by calling matlab session
        this part and the one below is to support PQN, the Proxiaml method is on its own, it does not need this
        """
        if model == "Lasso":
            return lambda u: Lasso(u, X, y, rho)
        elif model == "GroupLasso":
            return lambda u: GroupLasso(u, X, y, rho)
        elif model == "GeneralizedFusedLasso":
            return lambda u: GeneralizedFusedLasso(u, X, y, rho, L, mu)
        else:
            raise ValueError("Model not supported")
        
    def _define_projection(self, model, groups=None):
        """
        Support models: 'Lasso', GroupLasso', 'GeneralizedFusedLasso'
        """
        if model == "Lasso":
            return lambda u: ProjLassoGurobi(u, self.k, self.d)
        elif model == "GroupLasso":
            return lambda u: ProjGroupLassoGurobi(u, self.k, groups, self.h_selected, self.d)
        elif model == "GeneralizedFusedLasso":
            return lambda u: ProjGeneralizedFusedLassoGurobi(u, self.k, self.d)
        else:
            raise ValueError("Model not supported")
        
    def _max_degree(self, L):
        # find the maximum degree of the graph according to the laplacian matrix
        return np.max(np.diag(L.toarray()))
    
    def solver(self, model, X, y, clusters=None, L=None, A=None, i=None):
        if model == "Proximal":
            return self._solver_proximal(X, y, A, i)
        elif model == "Lasso":
            rho = np.sqrt(self.n) # TODO: check if this is the correct value
            funObj = self._define_obj(model, X, y, rho)
            funProj = self._define_projection(model)
        elif model == "GroupLasso":
            rho = np.sqrt(self.n) # TODO: check if this is the correct value
            funObj = self._define_obj(model, X, y, rho)
            funProj = self._define_projection(model, clusters)
        elif model == "GeneralizedFusedLasso":
            mu = 0.1
            max_degree = self._max_degree(L)
            rho = 3.4 * self.k * 2 * mu * 2 * max_degree  # TODO: check if this is the correct value
            funObj = self._define_obj(model, X, y, rho, L, mu)
            funProj = self._define_projection(model)
        else:
            raise ValueError("Model not supported")
        
        uSimplex = np.ones((self.d, 1)) / self.d
        uout, obj, _ = minConF_PQN(funObj, uSimplex, funProj, self.options)
        uout = funProj(uout) # TODO: check if this is necessary

        return uout
    
    def _call_proximal(self, datafile, resultfile, rho1, rho2):
        eng = matlab.engine.start_matlab()
        try:
            eng.cd(os.path.abspath('./code_fgfl_aaai14/'))
            eng.addpath(os.path.abspath('./code_fgfl_aaai14/GFL/'))
            eng.addpath(eng.genpath(os.path.abspath('./code_fgfl_aaai14/')))
            eng.gfl_proximal(datafile, resultfile, rho1, rho2, nargout=0)
        finally:
            eng.quit()

    def _cross_validation(self, X, y, A, rho1_values, rho2_values, k_folds=5):
        results = []
        n = self.n
        indices = np.arange(n)
        np.random.shuffle(indices)
        folds = np.array_split(indices, k_folds)
        datafile = os.path.join(self.datafile, 'data.mat') 
        resultfile = os.path.join(self.resultfile, 'result.mat')

        for rho1, rho2 in itertools.product(rho1_values, rho2_values):
            mse_list = []
            for fold in folds:
                train_indices = np.setdiff1d(indices, fold)
                test_indices = fold
                X_train, y_train = X[train_indices], y[train_indices]
                X_test, y_test = X[test_indices], y[test_indices]

                self._save_mat(X_train, y_train, A, datafile)
                self._call_proximal(datafile, resultfile, rho1, rho2)
                u, funcVal = self._read_result(resultfile)      
                mse = np.mean((X_test @ u - y_test) ** 2)
                mse_list.append(mse)

            avg = np.mean(mse_list)
            results.append((rho1, rho2, avg))

        best_rho1, best_rho2, _ = min(results, key=lambda x: x[2])
        return best_rho1, best_rho2

    def _read_result(self, resultfile):
        result = sio.loadmat(resultfile)
        beta, funcVal = result['beta'], result['funcVal']
        return beta, funcVal
        
    def _solver_proximal(self, X, y, A, i):
        # rho1_values = [0.1, 0.5, 1.0, 5.0]
        # rho2_values = [0.1, 0.5, 1.0, 5.0]
        # if i == 0: # we only do cross validation once and use the best rho1 and rho2 for the rest of the replications
        #     self.best_rho1, self.best_rho2 = self._cross_validation(X, y, A, rho1_values, rho2_values)
        #     print(f"Best rho1: {self.best_rho1}, Best rho2: {self.best_rho2}")

        datafile_name = os.path.join(self.datafile, f'data_{i}.mat')
        resultfile_name = os.path.join(self.resultfile, f'result_{i}.mat')
        self._save_mat(X, y, A, datafile_name)
        self._call_proximal(datafile_name, resultfile_name, self.best_rho1, self.best_rho2)
        u, funcVal = self._read_result(resultfile_name)
        return u.flatten() # the original return a vector with shape (d,1), will not work with recovery_accuracy

    def recovery_accuracy(self, u):
        # evaluate the support recovery accuracy
        # we take top k for u
        selected_features_true = np.arange(self.k)
        selected_features_pred = np.argsort(np.abs(u))[-self.k:] # take absolute value for proximal method
        correct_pred = np.intersect1d(selected_features_true, selected_features_pred)
        accuracy = len(correct_pred) / self.k
        return accuracy
    
    def _save_mat(self, X, y, A, filename=None):
        # save the data to .mat file so that the matlab code of proxiaml can use it
        # print("X.shape", X.shape)   
        # print("y.shape", y.shape)
        # print("A.shape", A.shape)
        if y.ndim == 1:
            y = y[:, np.newaxis]
        data = {
            "X": X,
            "y": y,
            "AdjMat": A.toarray() if sp.issparse(A) else A,  # we store the adjacency matrix as dense matrix
        }
        sio.savemat(filename, data)

    def _report(self, model_accuracy):
        for model, accuracy in model_accuracy.items():
            avg_accuracy = np.mean(accuracy)
            std_accuracy = np.std(accuracy)
            print(f"Model: {model}, Avg. Accuracy: {avg_accuracy}, Std. Accuracy: {std_accuracy}")

    def main(self):
        models = ["Proximal", "Lasso", "GroupLasso", "GeneralizedFusedLasso"]
        model_accuracy = defaultdict(list)
        for i in range(self.num_replications):
            L, w, X, y, clusters, A = self._generate_data()
            for model in models:
                print(f"Synthetic dataset {i+1}: Running {model} model")
                u = self.solver(model, X, y, clusters, L, A, i)
                accuracy = self.recovery_accuracy(u)
                model_accuracy[model].append(accuracy)

        self._report(model_accuracy)
        return model_accuracy
    
# TODO: solve the orginal problem and calculate the MSE?
    

In [14]:
from scipy.io import loadmat
data = loadmat('./code_fgfl_aaai14/result.mat')
beta = data['beta'].flatten()
k = 20
selected_features_true = np.arange(k)
selected_features_pred = np.argsort(np.abs(beta))[-k:] # take absolute value for proximal method
selected_features_pred = np.sort(selected_features_pred)
print("selected_features_pred", selected_features_pred)
correct_pred = np.intersect1d(selected_features_true, selected_features_pred)
accuracy = len(correct_pred) / k

selected_features_pred [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


In [18]:
a = RandomEnsemble(n=100, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.5, num_replications=50)
res = a.main()

Synthetic dataset 1: Running Lasso model
Iteration 6: Step size t = 0.21898357182107342, f_new = 0.004134819599850517, f_old = -0.0009770739991951253
Iteration 3: Step size t = 0.30922924749842606, f_new = 0.00013205826166264208, f_old = -0.00010663669043858344
Iteration 7: Step size t = 0.29755904988005744, f_new = 0.0014947178303513357, f_old = -8.946149510153062e-05
Iteration 5: Step size t = 0.28263006747855446, f_new = 3.719873639068964e-05, f_old = -0.00011054377325636855
Iteration 10: Step size t = 0.16007867968985245, f_new = 0.00018286080809598452, f_old = -0.0002451569349217464
Iteration 10: Step size t = 0.21490114741227528, f_new = 5.320921708991562e-05, f_old = -0.000274618723802427
Iteration 8: Step size t = 0.05065401708620121, f_new = 0.005483921872293285, f_old = -0.00018858656868587357
Iteration 10: Step size t = 0.06695005398798304, f_new = 0.0027482039986665447, f_old = -0.00016958444663503188
Iteration 9: Step size t = 0.06695105940910973, f_new = 0.024129918722099

In [19]:
# save res to a file
with open('res_1.pkl', 'wb') as f:
    pickle.dump(res, f)

In [20]:
# compute the mean with the confidence interval 95%
mean_accuracy = {k: np.mean(v) for k, v in res.items()}
std_accuracy = {k: np.std(v) for k, v in res.items()}
print(mean_accuracy)
print(std_accuracy)


{'Lasso': 0.20679999999999998, 'GroupLasso': 0.1672, 'GeneralizedFusedLasso': 0.6736, 'Proximal': 0.7832000000000002}
{'Lasso': 0.0527044590143946, 'GroupLasso': 0.08595440651880508, 'GeneralizedFusedLasso': 0.12396386570287327, 'Proximal': 0.19552432073785606}


In [48]:
a_10 = RandomEnsemble(n=10, d=100, k=5, h_total=5, h_selected=2, h_rest=3, gamma=0.1, p = 0.9, q = 0.05, num_replications=10)
res_10 = a_10.main()
print(res_10)

Synthetic dataset 1: Running Lasso model
Iteration 4: Step size t = 0.37294259665858565, f_new = 0.005260468674056384, f_old = -0.003330727156400086
Iteration 3: Step size t = 0.14371165777201067, f_new = 0.00020307352694875526, f_old = -0.0007266697450701607
Iteration 9: Step size t = 0.08527095214716107, f_new = 0.0003917674051733648, f_old = -6.590525632653232e-05
Synthetic dataset 1: Running GroupLasso model
Iteration 4: Step size t = 0.27401297980755224, f_new = 0.00014176894320392785, f_old = -0.00029000542329219266
Iteration 3: Step size t = 0.2497668867510191, f_new = 0.00021152374213065394, f_old = -0.00020083809931843586
Iteration 5: Step size t = 0.23347896520480893, f_new = 0.00017334380587458273, f_old = -8.945295703261297e-05
Iteration 9: Step size t = 0.1493815013271076, f_new = 0.00023071590771455637, f_old = -0.0001333032984064488
Iteration 5: Step size t = 0.08087740416410771, f_new = 0.0006447402121170522, f_old = -0.00012025015534419823
Synthetic dataset 1: Running 

In [49]:
mean_accuracy = {k: np.mean(v) for k, v in res_10.items()}
std_accuracy = {k: np.std(v) for k, v in res_10.items()}
print(mean_accuracy)
print(std_accuracy)


{'Lasso': 0.2, 'GroupLasso': 0.18, 'GeneralizedFusedLasso': 0.42000000000000004, 'Proximal': 0.14}
{'Lasso': 0.1264911064067352, 'GroupLasso': 0.10770329614269007, 'GeneralizedFusedLasso': 0.20880613017821098, 'Proximal': 0.23748684174075832}


In [51]:
a_11 = RandomEnsemble(n=10, d=100, k=5, h_total=5, h_selected=2, h_rest=3, gamma=0.1, p = 0.9, q = 0.05, num_replications=10)
res_11 = a_11.main()
print(res_11)

Synthetic dataset 1: Running Lasso model
Iteration 9: Step size t = 0.032115251451107274, f_new = 0.003477332753198389, f_old = -3.886572053477795e-05
Iteration 6: Step size t = 0.34305584658410804, f_new = 1.8795761551370304e-05, f_old = -8.899920817062691e-06
Iteration 9: Step size t = 0.0964813914517646, f_new = 0.00035358183931874905, f_old = -9.056880638261993e-05
Iteration 5: Step size t = 0.3145434953477083, f_new = 1.1222792814339314e-05, f_old = -1.9686522428613646e-05
Iteration 8: Step size t = 0.09367794553237296, f_new = 0.0005555050038181025, f_old = -4.263884716916764e-05
Iteration 4: Step size t = 0.3746689879113836, f_new = 1.0269601258451467e-05, f_old = -1.0809591170103228e-05
Iteration 7: Step size t = 0.04034866074452381, f_new = 0.004880265913656903, f_old = -4.471006994798456e-05
Iteration 8: Step size t = 0.11135712162733324, f_new = 0.00048403494510970333, f_old = -5.343182408400183e-05
Iteration 9: Step size t = 0.019493225949865245, f_new = 0.01396439152047912

In [52]:
mean_accuracy = {k: np.mean(v) for k, v in res_11.items()}
std_accuracy = {k: np.std(v) for k, v in res_11.items()}
print(mean_accuracy)
print(std_accuracy)


{'Lasso': 0.21999999999999997, 'GroupLasso': 0.18, 'GeneralizedFusedLasso': 0.5199999999999999, 'Proximal': 0.24000000000000005}
{'Lasso': 0.22715633383201095, 'GroupLasso': 0.18867962264113208, 'GeneralizedFusedLasso': 0.15999999999999998, 'Proximal': 0.233238075793812}


In [ ]:
a_2 = RandomEnsemble(n=50, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.1, p=0.9, q=0.05, num_replications=10)
res_2 = a_2.main()
print(res_2)

Synthetic dataset 1: Running Proximal model
1 3
Results saved to /Users/aolongli/Desktop/Research-GFL/Experiement/code/gfl/code_fgfl_aaai14/result_gfl/10_1000_50_20_5_15_0.1_0.9_0.05/result_0.mat
Synthetic dataset 1: Running Lasso model
Iteration 3: Step size t = 0.28707347434500385, f_new = 7.793311313082971e-05, f_old = -0.0002132737721788565
Iteration 3: Step size t = 0.29389359152845607, f_new = 0.00039946737313752086, f_old = -0.0001226230368442104
Synthetic dataset 1: Running GroupLasso model
Iteration 2: Step size t = 0.48870949682273424, f_new = 1.6974120453545688e-05, f_old = -1.208834039811957e-06
Synthetic dataset 1: Running GeneralizedFusedLasso model
Iteration 1: Step size t = 0.01789076556521685, f_new = 0.000639680558321655, f_old = 0.0
Synthetic dataset 2: Running Proximal model
1 3
Results saved to /Users/aolongli/Desktop/Research-GFL/Experiement/code/gfl/code_fgfl_aaai14/result_gfl/10_1000_50_20_5_15_0.1_0.9_0.05/result_1.mat
Synthetic dataset 2: Running Lasso model
I

In [67]:
random_ensemble = RandomEnsemble(n=50, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.1, p=0.9, q=0.05, num_replications=10)
random_ensemble.main()


Synthetic dataset 1: Running Proximal model
3 5
3 5
3 5
3 5
3 5
3 5
3 5
3 5
3 5
3 5
3 5
Results saved to /Users/aolongli/Desktop/Research-GFL/Experiement/code/gfl/code_fgfl_aaai14/result_gfl/50_1000_50_20_5_15_0.1_0.9_0.05/result_0.mat
Synthetic dataset 1: Running Lasso model
Iteration 3: Step size t = 0.3396583554423406, f_new = 0.00013197473433419413, f_old = -0.003091813404649368
Iteration 8: Step size t = 0.04083845874107539, f_new = 0.0047498836746327, f_old = -0.0002853789205263498
Iteration 6: Step size t = 0.329267739733213, f_new = 8.185284448208277e-06, f_old = -9.327079887599867e-05
Iteration 10: Step size t = 0.1622649315893243, f_new = 4.948864962489423e-05, f_old = -0.00018008186609363806
Iteration 10: Step size t = 0.23582054445434397, f_new = 3.290933716959603e-05, f_old = -0.00012947420489796725
Iteration 3: Step size t = 0.23796772827321055, f_new = 0.0001271345763233623, f_old = -5.850994045972811e-05
Iteration 6: Step size t = 0.20800759874357333, f_new = 3.26083736

In [ ]:
sample_sizes = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]
res = []
for n in sample_sizes:
    random_ensemble = RandomEnsemble(n=n, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.1, p=0.9, q=0.05, num_replications=10)
    model_accuracy = random_ensemble.main()
    res.append(model_accuracy)



In [27]:
a_3 = RandomEnsemble(n=100, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.5, num_replications=1)
res_3 = a_3.main()

Synthetic dataset 1: Running Lasso model
Iteration 6: Step size t = 0.2742908286087613, f_new = 0.00020271599488276126, f_old = -0.0009829343741030619
Iteration 6: Step size t = 0.3504018343795292, f_new = 0.00017820731654631574, f_old = -0.0004772365560986741
Iteration 6: Step size t = 0.1804439759997316, f_new = 0.0005562399473237915, f_old = -0.000570120610069896
Iteration 6: Step size t = 0.14835623380955154, f_new = 0.005686961164457991, f_old = -0.0004066318180697054
Iteration 8: Step size t = 0.07293834606031668, f_new = 0.00571524631670334, f_old = -0.0008826280779001207
Iteration 6: Step size t = 0.3863939544195375, f_new = 0.0005047952695672914, f_old = -7.143820236776757e-05
Iteration 6: Step size t = 0.17954970436034223, f_new = 0.003322103219421434, f_old = -0.00014889737824553802
Iteration 6: Step size t = 0.18354014899591786, f_new = 0.0027927128546186347, f_old = -8.195621632073987e-05
Iteration 6: Step size t = 0.29149646453654277, f_new = 0.0007999562243207004, f_old 

In [28]:
print(res_3) 

defaultdict(<class 'list'>, {'Lasso': [0.16], 'GroupLasso': [0.08], 'GeneralizedFusedLasso': [0.58], 'Proximal': [0.68]})


In [29]:
a_4 = RandomEnsemble(n=100, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.5, num_replications=5) # increase the rho
res_4 = a_4.main()
print(res_4)


Synthetic dataset 1: Running Lasso model
Iteration 6: Step size t = 0.23412133146384806, f_new = 0.0013067865645580848, f_old = -0.00040902226611800696
Iteration 6: Step size t = 0.24837434303967587, f_new = 0.00015614243056788588, f_old = -0.00019770492408082536
Iteration 6: Step size t = 0.2065248831262726, f_new = 0.0004273836528605414, f_old = -0.00019671211544730903
Iteration 6: Step size t = 0.24890638756100447, f_new = 9.884244106283618e-05, f_old = -0.00010904480815399386
Iteration 9: Step size t = 0.06365461579541498, f_new = 5.376014555909784e-05, f_old = -0.00020674501738371674
Iteration 6: Step size t = 0.2571603767833984, f_new = 0.00011350428935091389, f_old = -3.909000211943154e-05
Iteration 6: Step size t = 0.17235956666412744, f_new = 0.00017637470676746267, f_old = -5.961836970195034e-05
Iteration 9: Step size t = 0.18040511289962413, f_new = 3.809218399340145e-05, f_old = -8.606126289225361e-05
Iteration 9: Step size t = 0.09035463699457635, f_new = 0.000882497286231

In [30]:
print(res_4)

defaultdict(<class 'list'>, {'Lasso': [0.18, 0.16, 0.18, 0.36, 0.18], 'GroupLasso': [0.16, 0.08, 0.16, 0.34, 0.26], 'GeneralizedFusedLasso': [0.72, 0.7, 0.52, 0.74, 0.68], 'Proximal': [0.96, 0.82, 0.34, 0.68, 0.08]})


In [86]:
def add_noise_to_signal(signal, desired_snr_db):
    # Convert signal to numpy array
    signal = np.asarray(signal)
    # Calculate signal power
    signal_power = np.mean(signal ** 2)
    # Calculate the desired noise power based on the desired SNR (in dB)
    desired_snr_linear = 10 ** (desired_snr_db / 10)
    noise_power = signal_power / desired_snr_linear
    # Generate noise with the calculated power
    noise = np.random.normal(0, np.sqrt(noise_power), signal.shape)
    # Add noise to the original signal
    noisy_signal = signal + noise
    return noisy_signal, noise
 
 
def compute_snr(signal, noise):
    # Convert signal and noise to numpy arrays for calculations
    signal = np.asarray(signal)
    noise = np.asarray(noise)
 
    # Calculate the power of the signal and the noise
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    # Calculate the SNR
    snr = signal_power / noise_power
    # Convert SNR to decibels (dB)
    snr_db = 10 * np.log10(snr)
    return snr_db

def compute_snr(signal, noise):
    # Convert signal and noise to numpy arrays for calculations
    signal = np.asarray(signal)
    noise = np.asarray(noise)

    # Calculate the power of the signal and the noise
    signal_power = np.mean(signal ** 2)
    noise_power = np.mean(noise ** 2)
    
    # Calculate the SNR
    snr = signal_power / noise_power
    
    # Convert SNR to decibels (dB)
    snr_db = 10 * np.log10(snr)
    
    return snr_db



In [33]:
a_5 = RandomEnsemble(n=100, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.5, num_replications=1) # mu=0.1 was 1.0
res_5 = a_5.main()

Synthetic dataset 1: Running Lasso model
Iteration 6: Step size t = 0.23663009999452067, f_new = 0.0025530086903535146, f_old = -0.0013243206620230385
Iteration 6: Step size t = 0.25082838498229476, f_new = 0.0005025309897684865, f_old = -0.00022210057957459047
Iteration 6: Step size t = 0.10828927428517965, f_new = 0.0006953845031075865, f_old = -0.00025314871800134254
Iteration 6: Step size t = 0.14713312235797138, f_new = 0.00034966942158488066, f_old = -0.0001714743539078388
Iteration 8: Step size t = 0.09837331136296257, f_new = 0.00025348557828161597, f_old = -0.0003423550868950478
Iteration 6: Step size t = 0.23359112730044296, f_new = 0.0007379349562291356, f_old = -9.448650094695184e-05
Iteration 6: Step size t = 0.10349457264619222, f_new = 0.0008426507753128404, f_old = -0.00011540386723867868
Iteration 6: Step size t = 0.2591520669164413, f_new = 9.734106634431841e-05, f_old = -4.159211145162878e-05
Iteration 10: Step size t = 0.12341807766358237, f_new = 7.528357419764694e

In [34]:
res_5

defaultdict(list,
            {'Lasso': [0.14],
             'GroupLasso': [0.28],
             'GeneralizedFusedLasso': [0.72],
             'Proximal': [0.92]})

In [36]:
a_6 = RandomEnsemble(n=100, d=1000, k=50, h_total=20, h_selected=5, h_rest=15, gamma=0.5, num_replications=10) # mu=0.1 was 1.0
res_6 = a_6.main()

Synthetic dataset 1: Running Lasso model
Iteration 6: Step size t = 0.14859713761966464, f_new = 0.009823750281762663, f_old = -0.000995608014109134
Iteration 6: Step size t = 0.20003148187861963, f_new = 0.011183308947061998, f_old = -0.0004891473433656389
Iteration 8: Step size t = 0.047865362514252, f_new = 0.001475047283068953, f_old = -0.00022292627855311154
Iteration 6: Step size t = 0.1970388734553259, f_new = 0.0005746425580758716, f_old = -4.3573325957984546e-05
Iteration 6: Step size t = 0.06781572918876333, f_new = 0.000979655140318868, f_old = -6.459462156995268e-05
Iteration 6: Step size t = 0.11267869638960903, f_new = 4.675023235098337e-05, f_old = -5.697125447596517e-05
Iteration 9: Step size t = 0.022360275180694278, f_new = 0.009974033680031547, f_old = -0.0002730819123729662
Iteration 6: Step size t = 0.1750063920296937, f_new = 8.511902469313104e-05, f_old = -4.2487672177561646e-05
Iteration 10: Step size t = 0.1631011567706696, f_new = 6.558663531398216e-05, f_old 

In [37]:
print(res_6)

defaultdict(<class 'list'>, {'Lasso': [0.28, 0.26, 0.14, 0.12, 0.24, 0.18, 0.12, 0.32, 0.26, 0.22], 'GroupLasso': [0.14, 0.24, 0.0, 0.0, 0.28, 0.0, 0.12, 0.2, 0.24, 0.12], 'GeneralizedFusedLasso': [0.78, 0.66, 0.44, 0.44, 0.8, 0.46, 0.64, 0.72, 0.6, 0.56], 'Proximal': [0.92, 0.98, 0.24, 0.5, 0.88, 0.26, 0.82, 0.9, 0.94, 0.96]})


# Section 2: Model Optimization

Here we include the all method for the optimization problem. We will use the following optimization methods to solve the problem:
- Lasso
- NeuIPS 2021
- Fast GFL AAAI-2014
- Our method: Boolean GFL

# Section 3: Experiments

Here we do the experiments: fixed the random ensemble parameters, we generate the data and run the optimization methods. We compare the performance of the methods by the top-k support recovery accuracy and out-of-sample MSE.

We will start with small sample size and increase the sample size with other fixed parameters. For each parameter, we will generate 100 random ensemble and report the average performance in the end. For method with hyperparameters, we use cross-validation to select the best hyperparameters.

# Section 4: Plot

Here we plot the result with respect to the sample size. The error bar means 95% confidence interval.